In [ ]:
# Import stuff
from os.path import join
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.interfaces.fsl.utils import Merge, ImageMeants
from nipype.interfaces.fsl.model import Randomise, Cluster
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.fsl.maths import ApplyMask
from pandas import DataFrame, Series

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
#analysis_home = '/Users/catcamacho/Box/LNCD_rewards_connectivity'
analysis_home = '/Volumes/Zeus/Cat'
#raw_dir = analysis_home + '/subjs'
raw_dir = '/Volumes/Phillips/bars/APWF_bars/subjs'
preproc_dir = analysis_home + '/proc/preprocessing'
firstlevel_dir = analysis_home + '/proc/firstlevel'
secondlevel_dir = analysis_home + '/proc/secondlevel'
workflow_dir = analysis_home + '/workflows'
template_dir = analysis_home + '/templates'
MNI_template = template_dir + '/MNI152_T1_1mm_brain.nii'

#pull subject info 
subject_info = DataFrame.from_csv(analysis_home + '/misc/subjs_all.csv')

conditions = ['punish','neutral']
seed_names = ['L_amyg','R_amyg']

# Group analysis models (predicting FC)


In [ ]:
## Functions for mixed effect linear modeling

# LMEM for MRI data (3D nifti data)
def mri_lmem(model, subject_dataframe, subject_files, grouping_variable):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from sys import stdout
    from os import getcwd
    import statsmodels.formula.api as smf
    from nibabel import load, save
    from numpy import array, empty_like, stack
    
    working_dir = getcwd() + '/'
    model = 'brain ~ age + sex + age*sex' 
    
    # set up to print output to a text file
    orig_stdout = sys.stdout
    out_file = open( working_dir + 'model_summary.txt','w')
    sys.stdout = out_file
    
    # Load the brain data
    brain_niftis = [load(brain) for brain in subject_files]
    brain_data = [brain.get_data() for brain in brain_niftis]
    brain_data_4D = stack(brain_data, axis=3)
    
    ## Preallocate the output arrays
    # for the model
    ICCs_data = empty_like(brain_data[0])
    pval_intercept_data = empty_like(brain_data[0])
    pval_age_data = empty_like(brain_data[0])
    pval_sex_data = empty_like(brain_data[0])
    pval_ageSexInteract_data = empty_like(brain_data[0])
    # per subject
    residuals_data = empty_like(brain_data_4D)
    fe_params_data = empty_like(brain_data_4D)
    re_params_data = empty_like(brain_data_4D)
    
    # Set up the actual loops
    
    
    # Run the mixed effects linear model
    mlm = smf.mixedlm(model, data, groups=data[grouping_variable])
    fitmodel = mlm.fit()
    print(fitmodel.summary())
    
    out_file.close()
    sys.stdout = orig_stdout
    
    # Save the ouputs as nifti files
    output_data1 = [ICCs_data, pval_intercept_data, pval_invAge_data,
                    pval_sex_data, pval_ageSexInteract_data]
    output_data2 = [residuals_data, fe_params_data, re_params_data]
    output_niftis1 = [Nifti1Image(result, brain_niftis[0].affine) for result in output_data1]
    ###### I don't think this one will work... test #######
    output_niftis2 = [Nifti1Image(result, brain_niftis.affine) for result in output_data2]
    output_niftis = output_niftis1 + output_niftis2
    
    output_filenames = ['ICCs.nii','pval_intercept_data.nii','pval_age_data.nii',
                        'pval_sex_data.nii','pval_ageSexInteract_data.nii',
                        'residuals_data.nii','fe_params_data.nii','re_params_data.nii']
    for a in output_niftis:
        save(a, working_dir + output_filenames(a.index))
    
    return(output_files)

In [ ]:
## Data handling nodes

conditionsource = Node(IdentityInterface(fields=['condition','seed']),
                       name='conditionsource')
conditionsource.iterables = [('condition',conditions),('seed', seed_names)]

# Grab the subject beta maps 
time_template = {'beta_maps':firstlevel_dir + '/smoothedMNI_conn_beta/*/%s/%s/betas_flirt_smooth_masked.nii'}
betamap_grabber = Node(DataGrabber(sort_filelist=True,
                                   field_template = time_template,
                                   base_directory=firstlevel_dir,
                                   template=firstlevel_dir + '/smoothedMNI_conn_beta/*/%s/%s/betas_flirt_smooth_masked.nii',
                                   infields=['condition','seed'],
                                   template_args={'beta_maps':[['condition','seed']]}), 
                       name='betamap_grabber')

# Sink relavent data
substitutions = [('_condition_',''),
                 ('_seed_','')]
datasink = Node(DataSink(substitutions=substitutions, 
                         base_directory=secondlevel_dir,
                         container=secondlevel_dir), 
                name='datasink')

In [ ]:
## Analysis nodes

# merge beta maps into one file
merge = Node(Merge(dimension = 't'),name='merge')


# Cluster the results
cluster_results = MapNode(Cluster(threshold=2,
                                  out_index_file=True,
                                  out_localmax_txt_file=True),
                          name='cluster_results', 
                          iterfield = ['in_file'])

In [ ]:
groupanalysisflow = Workflow(name='groupanalysisflow')
groupanalysisflow.connect([(conditionsource, betamap_grabber, [('condition','condition'),
                                                               ('seed','seed')]),
                           (betamap_grabber, merge, [('beta_maps','in_files')]),
                           (mask_tstat, cluster_results, [('out_file','in_file')]),
                           (cluster_results, datasink, [('index_file','cluster_index_file'), 
                                                        ('localmax_txt_file','cluster_localmax_txt_file')])
                          ])
groupanalysisflow.base_dir = workflow_dir
groupanalysisflow.write_graph(graph2use='flat')
groupanalysisflow.run('MultiProc', plugin_args={'n_procs':24})